In [1]:
import llm_core.llm as L
import re

In [2]:
llm = L.LLM('You are a statistic and fact sharing bot.')

s = llm(
    'What are some country capitals?',
    response_format={'country': str, 'capital': str, 'pop': int},
    max_tokens=1000,
    temperature=0.1
)

for t in s:
    print(t, end='')

Initializing Global LLM Instance


ValueError: too many values to unpack (expected 2)

In [22]:
import json
import re

class Repr:
    def __init__(self, s): self.s = s
    def __str__(self): return self.s
    def __repr__(self): return self.s

def pretty_response_format(response_format):
    if response_format in [str, int, float]:
        return Repr(response_format.__name__)
    elif type(response_format) == dict:
        return {
            Repr(f'"{k}"'): pretty_response_format(v)
            for k,v in response_format.items()
        }
    elif type(response_format) == list:
        sub, etc = response_format
        assert etc == ...

        return [pretty_response_format(sub), Repr('...')]
    else:
        raise Exception(f'Unsupported value type: {v}')

# num_grammar = L.LlamaGrammar.from_string(
#     r'root ::= ("-"? ([0-9] | [1-9] [0-9]*)) ("." [0-9]+)? ([eE] [-+]? [0-9]+)? [}.]',
#     verbose=False
# )

# str_grammar = L.LlamaGrammar.from_string(
#     r'''root ::= 
#         (
#             [^\\] |
#             "\\" (["\\/bfnrt] | "u" [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F]) # escapes
#         )*
#     ''',
#     verbose=False
# )

# fmt = {'country': str, 'capital': str, 'pop': int}
# fmt = {'country': str, 'capital': str, 'pop': int, 'local_names': [str, ...]}
fmt = {'country': str, 'capital': str, 'pop': int, 'nearest_country': {'name': str, 'pop': int}}
# fmt = {'country': str, 'capital': str, 'pop': int, 'nearest_cities': [{'name': str, 'pop': int}, ...]}

fmt_prompt = pretty_response_format(fmt)

# print(str(fmt_prompt))
# print(f'{fmt_prompt}')
# assert 0

def gen_response_formated_rec(response_format, prompt_ref, temperature=0, max_tokens=1000):
    if response_format == str:
        prompt_ref[0] += '"'
        s = L.LLM_GLOBAL_INSTANCE(
            prompt_ref[0],
            max_tokens=max_tokens, temperature=temperature,
            stream=True,
            # grammar=str_grammar
        )
        for t in s:
            t = t['choices'][0]['text']
            print(t)
            str_end = re.search(r'(^|[^\\])(")', t)
            if str_end:
                prompt_ref[0] += t[:str_end.span(2)[0]]
                break
            prompt_ref[0] += t
        prompt_ref[0] += '"'
    elif response_format in [float, int]:
        s = L.LLM_GLOBAL_INSTANCE(
            prompt_ref[0],
            max_tokens=max_tokens, temperature=temperature,
            stream=True,
            # grammar=num_grammar
        )
        for t in s:
            t = t['choices'][0]['text']
            print(t)
            str_end = re.search(r'[,}]', t)
            if str_end:
                prompt_ref[0] += t[:str_end.span()[0]]
                break
            prompt_ref[0] += t
    elif type(response_format) == dict:
        prompt_ref[0] += '{'
        first = True
        for k,v in response_format.items():
            if not first: prompt_ref[0] += ', '
            first = False
            prompt_ref[0] += f'"{k}": '
            gen_response_formated_rec(v, prompt_ref, temperature, max_tokens)
        prompt_ref[0] += '}'
    elif type(response_format) == list:
        assert 0

def gen_response_formated(response_format, prompt, temperature=0, max_tokens=1000):
    json_start = len(prompt)
    prompt_ref = [prompt]
    gen_response_formated_rec(fmt, prompt_ref, temperature, max_tokens)
    return prompt_ref[0][json_start:]

prompt = f'[INST] What is one capital city in Europe? Respond in JSON with this format ending strings with double quotes and numbers with JS notation (commas end rather than separate digits): {fmt_prompt} [/INST] '


resp = gen_response_formated(fmt, prompt)

print(resp)

assert 0

json_start = len(prompt)
prompt += '{'


first = True
for k,v in fmt.items():
    if not first: prompt += ', '
    first = False
    prompt += f'"{k}": '
    if v == str:
        prompt += '"'
        s = L.LLM_GLOBAL_INSTANCE(
            prompt,
            max_tokens=1000, temperature=0,
            stream=True,
            grammar=str_grammar
        )
        for t in s:
            t = t['choices'][0]['text']
            str_end = re.search(r'(^|[^\\])(")', t)
            if str_end:
                prompt += t[:str_end.span(2)[0]]
                break
            prompt += t
        prompt += '"'
    elif v in [int, float]:
        s = L.LLM_GLOBAL_INSTANCE(
            prompt,
            max_tokens=1000, temperature=0,
            stream=True,
            grammar=num_grammar
        )
        for t in s:
            t = t['choices'][0]['text']
            str_end = re.search(r'[,}]', t)
            if str_end:
                prompt += t[:str_end.span()[0]]
                break
            prompt += t
    else:
        raise Exception(f"Unsupported value type: {v}")
prompt += '}'

resp = json.loads(prompt[json_start:])

print(resp)

Fr
ance
",
Par
is
",
2
1
4
8
0
0
0
0
,
G
erm
any
",
8
3
1
4
9
3
0
0
}}
{"country": "France", "capital": "Paris", "pop": 21480000, "nearest_country": {"name": "Germany", "pop": 83149300}}


AssertionError: 

In [30]:
llm = L.LLM('You are a Finnish language teacher teaching Finnish to me through conversation. You must give accurate information.')

In [31]:
s = llm('Oletko koira? respond in English.', response_format='stream', max_tokens=1000, temperature=0.1)
for t in s:
    print(t, end='')

"Are you a dog?" the answer would be no, since I am a language teacher and not a dog. In Finnish, the question "Oletko koira?" would translate to "Are you a dog?" and the answer in Finnish would be "En ole koira."

In [6]:
s = llm('Why did you start your response with an extra space????? AHHHH!!', response_format='stream', max_tokens=1000, temperature=0)
for t in s:
    print(t, end='')

 I apologize for the extra space in my previous response. It was an accident and I will make sure it doesn't happen again. I understand how it can be confusing and I apologize for any inconvenience it may have caused.

Please let me know if you have any questions about the Finnish language, and I will be happy to help. I will make sure to provide accurate and standard Finnish information. Thank you for your understanding.

In [41]:
# llm._hist_to_prompt()
L.LLM.detokenize(llm._hist_to_prompt())
# print(L.LLM.detokenize(llm._hist_to_prompt()))

"[INST] You are a statistic and fact sharing bot. [/INST] Understood.  [INST] What be the captl of Finlan?. [/INST] The capital of Finland is Helsinki. It is the country's most populous city and plays an important role in political, educational, cultural, and economic life in Finland. Helsinki was founded in 1550 by King Gustav I of Sweden and was originally intended to be a rival to the Hanseatic city of Tallinn in Estonia. Today, Helsinki is a modern, bustling city with a high standard of living and a rich cultural scene."

In [8]:
llm._hist

[Msg(role='system', content='System: You are a very British assistant, and you make your British-ness clear in every response by phonetically spelling EVERY WORD in "Bri\'ish". Yew mahst speal wourds laik thies.', response_format=None),
 Msg(role='user', content='Ello thar mate', response_format=None),
 Msg(role='assistant', content=" Hello there! It's nice to see you. How can I assist you today?", response_format=None),
 Msg(role='user', content='Ya gaht them goud croumpets?!?!', response_format=None),
 Msg(role='assistant', content=' Yes, we do have some delicious crumpets! Would you like me to toast them for you and serve with butter and jam?', response_format=None)]

In [7]:
ts = L.LLM_GLOBAL_INSTANCE.tokenize(b'[INST] You are a Finnish language teacher strictly teaching standard Finnish. You must give accurate information. [/INST] Understood.', add_bos=False)
ts = [1, *ts, 2]
ts += L.LLM_GLOBAL_INSTANCE.tokenize(' [INST] What does "mitä on sinun nimi" mean? 1) What is another way to say it? 2) Why is the original version NOT grammatically correct? Tell me in terms of every individual word. [/INST] '.encode('utf-8'), add_bos=False)

In [10]:

L.LLM.detokenize(ts)

'[INST] You are a Finnish language teacher strictly teaching standard Finnish. You must give accurate information. [/INST] Understood.  [INST] What does "mitä on sinun nimi" mean? 1) What is another way to say it? 2) Why is the original version NOT grammatically correct? Tell me in terms of every individual word. [/INST] '

In [8]:
resp = L.LLM_GLOBAL_INSTANCE(ts, temperature=0, max_tokens=1000, stream=True)
for tok in resp:
    print(tok['choices'][0]['text'], end='')

1) "Mitä on sinun nimesi?" is one way to say "What is your name?" in standard Finnish. This sentence is in the passive form, which is a more formal way to ask someone's name in Finnish.
2) The original version "Mitä on sinun nimi" is not grammatically correct because it is missing the genitive case ending "-si" in the word "nimi". In Finnish, when asking someone's name, the noun "nimi" (name) should be in the genitive case and take the ending "-si". So, the correct form is "Mitä on sinun nimesi?"

To break it down further:

* "Mitä" is a pronoun and it means "what".
* "on" is the third person singular

KeyboardInterrupt: 

In [13]:
resp['choices'][0]['text']

" Kaitlyn, but you can call me Kait. I'm a 20-something year old girl from the Midwest. I'm a lover of all things fashion, beauty, and lifestyle. I'm a lover of all things pink, glitter, and sparkle. I'm a lover of all things that make me happy. I'm a lover of all things that make me feel good. I'm a lover of all things that make me feel"

In [8]:
s = llm('Hello. How are yo?', response_format={'my_name':'...', 'my_response':'...'}, max_tokens=1000)
s

{'my_name': 'Assistant',
 'my_response': "I'm here to help answer any questions you might have in a friendly British manner. How may I assist you today?"}

In [4]:
print(L.LLM.detokenize(llm._hist_to_prompt()))

[INST] You are a very British assistant, and you make your British-ness clear in every response by phonetically spelling EVERY WORD in British. [/INST] Understood. [INST] Good morning. [/INST]   Good mourning.

(Note: In British English, the word "morning" is pronounced with a long "o" sound, as in "mow-ning.") 


In [ ]:
$$$  TODO ^ remove the "respond in json" instruction in history

In [2]:
def logits_processor(prev_tok_ids, next_tok_logits):
    
    # print(prev_tok_ids, next_tok_logits)

    next_tok_logits[995] -= 1000
 
    return next_tok_logits

In [8]:
# llm = L.LLM('You are a British butler. Start every answer with "you"')
llm = L.LLM('You are a British assistant.')
resp = llm('Hello. What is the capital of Finland?', max_tokens=200, logits_processor=[logits_processor], response_format={'answer_reason':'...', 'answer':'...'})
resp

{'answer_reason': 'Helsinki is the capital city of Finland.',
 'answer': 'Helsinki'}

In [9]:
print(llm.get_pretty_hist())

system --- You are a British assistant.
__________

user --- Hello. What is the capital of Finland?
__________

assistant --- {'answer_reason': 'Helsinki is the capital city of Finland.', 'answer': 'Helsinki'}
__________




In [43]:
llm.tokenize(' You')

[28705, 995]

In [44]:
llm.detokenize([1, 995])

'You'